In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.metrics import RootMeanSquaredError
from time import time
from sklearn.metrics import mean_absolute_error
import wandb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In my previous post, I went through the following process:

Environment: Anaconda, Windows 11.

- Data wrangling (Light exploration, followed by removing and transforming some variables)
- Split the training dataset into training and validation datasets
- Fit a model using a Scikit-Learn pipeline (Data Preprocessing + fitting XGBoost/LightGBM estimators with a Randomized Search across their respective hyperparameters)
- Evaluate and visualize model performance
- Implement an automated approach to selecting hyperparameters (HyperOpt)
- Make predictions

In this post, I will implement the following using the same wrangled/preprocessed data:

Environment: Docker, Windows Subsystem for Linux 2 (WSL 2), Windows 11.

- Build a simple Sequential model in Keras/Tensorflow
- Use the Weights and Biases (WandB) platform to select optimal hyperparameters and record experiments.
  - Experiments are evaluated using K-Fold Cross Validation. Mean RMSE across folds for each experiment are custom logged in WandB. 
- Make predictions.
- Blend predictions from the previous post (decision tree) and this post (neural net).
  - By both taking the mean of predictions, and defining a meta-model trained on a holdout dataset kept completely separate.

In [4]:
# Function to bring in wrangled/preprocessed data from previous post
def data():
    training = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/sklearn/training_preprocessed")
    validation = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/sklearn/validation_preprocessed")
    holdout = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/sklearn/holdout_preprocessed")
    holdout_predictions_df = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/sklearn/holdout_preds_preprocessed")
    test = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/sklearn/test_preprocessed")
    
    X_train = training.drop(columns="SalePrice")
    y_train = training["SalePrice"]
    X_valid = validation.drop(columns="SalePrice")
    y_valid = validation["SalePrice"]
    X_holdout = holdout.drop(columns="Actual_SalePrice")
    y_holdout = holdout["Actual_SalePrice"]
    X_test = test
    holdout_predictions_df = holdout_predictions_df
    return X_train, y_train, X_valid, y_valid, X_holdout, y_holdout, X_test, holdout_predictions_df

In [5]:
# Bring in data
X_train, y_train, X_valid, y_valid, X_holdout, y_holdout, X_test, holdout_predictions_df = data()

In [63]:
# Log transforming SalePrice
y_list = [y_train, y_valid, y_holdout] 
y_train, y_valid, y_holdout = [np.log(y) for y in y_list]

In [6]:
# Since this model uses k-fold validation, we don't need separate training and validation datasets
X_train = X_train.append(X_valid).reset_index().drop(columns="index")
y_train = y_train.append(y_valid).reset_index().drop(columns="index").values

<ipython-input-6-33faa622c081>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train = X_train.append(X_valid).reset_index().drop(columns="index")
<ipython-input-6-33faa622c081>:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_train = y_train.append(y_valid).reset_index().drop(columns="index").values


In [7]:
# Log into Weights and Biases
wandb.init(project="house-price-prediction", entity="luiscostigan")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: luiscostigan (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [75]:
# Define simple Sequential model
def create_model():
    model = Sequential()
    model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(wandb.config.dense1, activation='relu'))
    model.add(Dropout(wandb.config.dropout1))
    model.add(Dense(wandb.config.dense2, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss="mse", metrics=[RootMeanSquaredError()])
    
    return model

In [81]:
# Define training function and hyperparameter ranges
from wandb.keras import WandbCallback
from sklearn.model_selection import KFold

sweep_config = {
  "name": "keras-sequential-model-sweep",
  "method": "random",
  "parameters": {
    "dropout1": {
      "min": 0.0,
      "max": 0.3
    },
    "dense1": {
      "values": [8, 16, 32]
    },
    "dense2": {
      "values": [8, 16, 32]
    },
    "epochs": {
      "values": [250,500,750]
    },
    "batch_size": {
      "values": [8, 16, 64, 128]
    }
  },
  "metric": {
    "name": "Mean Validation RMSE (all folds)",
    "goal": "minimize"
  }
}

config_defaults = {
  "dropout1": 0.1,
  "dense1": 16,
  "dense2": 16,
  "epochs": 100,
  "batch_size": 16
}

# Define number of splits
kf = KFold(n_splits=5)

def train():

  rmse_per_fold = []
  loss_per_fold = []
  fold_no = 1

  # Go through each split, and get the index number for each (currently 50, 10 x 5 repeated k-fold) 
  for train, test in kf.split(X_train, y_train):

    # With the current session in WandB
    with wandb.init(config=config_defaults) as run:

      # Recreate the model each time for each new batch
      model = None # Not sure if this step is necessary
      model = create_model()

      # Fit model on new batches
      model.fit(
        np.asarray(X_train), 
        y_train, 
        epochs=wandb.config.epochs, 
        batch_size=wandb.config.batch_size, 
        verbose=0,
        callbacks=[WandbCallback()], 
        validation_data=(np.asarray(X_train),y_train)
        )
      
      # Generate data for each
      scores = model.evaluate(np.asarray(X_train), y_train, callbacks=[WandbCallback()])
      print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]}')
      rmse_per_fold.append(scores[1])
      loss_per_fold.append(scores[0])

      # Increase fold number
      fold_no = fold_no + 1

      # == Provide average scores ==
      print('------------------------------------------------------------------------')
      print('Score per fold')
      for i in range(0, len(rmse_per_fold)):
        print('------------------------------------------------------------------------')
        print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - RMSE: {rmse_per_fold[i]}%')
      print('------------------------------------------------------------------------')
      print('Average scores for all folds:')
      print(f'> RMSE: {np.mean(rmse_per_fold)} (+- {np.std(rmse_per_fold)})')
      print(f'> Loss: {np.mean(loss_per_fold)}')
      print('------------------------------------------------------------------------')

      wandb.log({
        "Mean Validation RMSE (all folds)": np.mean(rmse_per_fold),
        "Mean Validation Loss (all folds)": np.mean(loss_per_fold) 
        })

keras_sequential_sweep_1 = wandb.sweep(sweep_config, project="house-price-prediction", entity="luiscostigan")

count = 20

wandb.agent(keras_sequential_sweep_1, function=train, count=count)

Create sweep with ID: hbftehhf
Sweep URL: https://wandb.ai/luiscostigan/house-price-prediction/sweeps/hbftehhf


wandb: Agent Starting Run: tjv4e6i3 with config:
wandb: 	batch_size: 16
wandb: 	dense1: 8
wandb: 	dense2: 16
wandb: 	dropout1: 0.07939069771740113
wandb: 	epochs: 250
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


37/37 [==============================] - 0s 2ms/step - loss: 0.4211 - root_mean_squared_error: 0.6490
Score for fold 1: loss of 0.42114102840423584; root_mean_squared_error of 0.6489537954330444
------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.42114102840423584 - RMSE: 0.6489537954330444%
------------------------------------------------------------------------
Average scores for all folds:
> RMSE: 0.6489537954330444 (+- 0.0)
> Loss: 0.42114102840423584
------------------------------------------------------------------------


Mean Validation Loss (all folds),▁
Mean Validation RMSE (all folds),▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
root_mean_squared_error,█▃▃▂▂▂▂▁▁▂▁▁▁▂▁▂▁▁▁▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▁▂▄▇▄▃▃▆▆▁▅▆█▂▄▅▃▃▃▅▂▃▄▅▂▂▁▃▂▁▂▃▃▂▄▄▃▄▄▂
val_root_mean_squared_error,▁▃▅▇▄▃▄▆▆▂▆▇█▂▅▆▄▃▃▆▃▄▅▆▃▂▂▄▃▂▂▄▄▃▅▅▄▅▄▃
Mean Validation Loss (all folds),0.42114
Mean Validation RMSE (all folds),0.64895
best_epoch,4
best_val_loss,0.08593


wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


...

Mean Validation Loss (all folds),▁
Mean Validation RMSE (all folds),▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
root_mean_squared_error,█▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▂▇█▃▃▄▃▃▃▂▂▂▂▂▂▂▂▂▃▂▂▁▂▂▂▁▁▁▁▂▂▂▂▂▁▁▂▁▁▁
val_root_mean_squared_error,▂██▃▄▄▄▃▄▃▂▃▂▂▂▃▃▂▃▂▃▁▂▂▂▂▂▂▁▂▂▂▂▂▂▁▂▂▂▂
Mean Validation Loss (all folds),0.84293
Mean Validation RMSE (all folds),0.83806
best_epoch,1
best_val_loss,0.44117


wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


37/37 [==============================] - 0s 2ms/step - loss: 0.1791 - root_mean_squared_error: 0.4232
Score for fold 5: loss of 0.17911989986896515; root_mean_squared_error of 0.4232255816459656
------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.055125754326581955 - RMSE: 0.23478874564170837%
------------------------------------------------------------------------
> Fold 2 - Loss: 1.4297456741333008 - RMSE: 1.1957197189331055%
------------------------------------------------------------------------
> Fold 3 - Loss: 1.2161591053009033 - RMSE: 1.1027960777282715%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.6706714630126953 - RMSE: 0.8189453482627869%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.17911989986896515 - RMSE: 0.4232255816459656%
----------------------------

Mean Validation Loss (all folds),▁
Mean Validation RMSE (all folds),▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
root_mean_squared_error,█▃▃▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▅█▆▅▅▄▅▃▄▄▂▂▁▂▂▂▂▂▂▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_root_mean_squared_error,▆█▆▆▆▅▆▄▅▅▃▃▂▂▂▃▃▂▂▂▃▂▂▂▂▂▁▂▂▁▁▁▂▁▂▁▂▁▁▁
Mean Validation Loss (all folds),0.71016
Mean Validation RMSE (all folds),0.7551
best_epoch,432
best_val_loss,0.10497


wandb: Agent Starting Run: ib9wcgoe with config:
wandb: 	batch_size: 16
wandb: 	dense1: 32
wandb: 	dense2: 8
wandb: 	dropout1: 0.1358070817330264
wandb: 	epochs: 250
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Hyperparameter optimization and experiment recording all took place within the Weights and Biases platform. The set of hyperparametrs resulting in the lowest loss (in terms of RMSE) is noted in the top row of the image below:

<img src="./wandb_rmse.png" width="800">

In [77]:
# Enter best params from sweep
best_params = {
    "dropout": 0.1729,
    "dense1": 8,
    "dense2": 8,
    "epochs": 200,
    "batch_size": 8
}

# Build model using the best parameters
def make_predictions(best_params, dataset):
    
    model = Sequential()
    model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(best_params.get("dense1"), activation='linear'))
    model.add(Dropout(best_params.get("dropout")))
    model.add(Dense(best_params.get("dense2"), activation='linear'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss="mse", metrics=[RootMeanSquaredError()])
              
    model.fit(X_train, y_train, epochs=best_params.get("epochs"),
              batch_size=best_params.get("batch_size"), verbose = 0)
    
    preds  = model.predict(dataset, best_params.get("batch_size"), verbose = 0)
              
    return preds

In [80]:
np.log(test_predictions)

array([[11.663364],
       [12.045205],
       [12.201768],
       ...,
       [12.03668 ],
       [11.636952],
       [12.296822]], dtype=float32)

In [78]:
# Make predictions
test_predictions = make_predictions(best_params, X_test)
holdout_predictions = make_predictions(best_params, X_holdout)

# Undo the log transform
test_predictions = np.exp(test_predictions)
holdout_predictions = np.exp(holdout_predictions)

# Generating submission CSV
d = {"Id":X_test.index,"SalePrice":test_predictions.flatten()}
submission = pd.DataFrame(data=d, index=None)

submission["Id"] = submission["Id"] + 1461

submission.to_csv("submission_nn.csv",index=False)

## Blending Predictions

So far, I have generated predictions using a decision tree-based model and a neural net-based model.
First, I'll try taking the mean of predictions from both to see how it performs.

In [38]:
# Read CSVs
decision_tree_predictions = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/sklearn/submission_dt.csv")
neural_net_predictions = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/tensorflow/submission_nn.csv")

# Merge CSVs on Id column
decision_tree_predictions["neural_net_predictions"] = neural_net_predictions["SalePrice"]

# Rename decision tree predictions column
decision_tree_predictions = decision_tree_predictions.rename(columns={"SalePrice":"decision_tree_predictions"})

# Add new column with mean
decision_tree_predictions["SalePrice"] = decision_tree_predictions[["neural_net_predictions","decision_tree_predictions"]].mean(axis=1)

# Drop other columns
decision_tree_predictions = decision_tree_predictions[["Id","SalePrice"]]

# Create mean prediction submission CSV
mean_predictions = decision_tree_predictions
mean_predictions.to_csv("submission_mean.csv",index=False)

Taking the mean of predictions did not beat my score from just using LightGBM in the previous post.

<img src="./mean_predictions_kaggle.png" width="600">

Next, I'll try defining a meta-model to best blend predictions from the two models. After developing each model, predictions were also made on a holdout dataset that was kept separate from the training and validation datasets, for the explicit purpose of training this meta-model. The model trained on this dataset was then used to blend predictions on the test dataset to be submitted to Kaggle.

In [39]:
# Append holdout set NN predictions to DT predictions
holdout_predictions_df["NN_predictions"] = holdout_predictions
holdout_predictions_df = holdout_predictions_df[["DT_predictions", "NN_predictions", "Actual_SalePrice"]]

In [40]:
# Get predictions on test set
test_predictions_dt = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/sklearn/submission_dt.csv")
test_predictions_nn = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/tensorflow/submission_nn.csv")

# Merge CSVs on Id column
test_predictions_dt["NN_predictions"] = test_predictions_nn["SalePrice"]

# Rename decision tree predictions column
test_predictions_dt = test_predictions_dt.rename(columns={"SalePrice":"DT_predictions"})

# Rename df
blended_predictions_df = test_predictions_dt

The meta-model was a simple grid search across different estimators, without attempting to optimize hyperparameters. Since the holdout dataset is very small, I used 10 folds in the cross-validation process.

In [41]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, mean_squared_error
import xgboost as xgb
import lightgbm as lgb

# Defining a custom loss function (Root Mean Squared Error)
rmse = make_scorer(mean_squared_error, squared=False)

X = holdout_predictions_df[["DT_predictions", "NN_predictions"]]
y = holdout_predictions_df["Actual_SalePrice"]

estimators = [
    {
        "clf": (LinearRegression(),)
    },
    {
        "clf": (Ridge(),)
    },
    {
        "clf": (xgb.XGBRegressor(),)
    },
    {
       "clf": (lgb.LGBMRegressor(),)
    }
]

pipe = Pipeline([("clf", LinearRegression())])

grid_search = GridSearchCV(pipe, estimators, cv=10, scoring=rmse)
grid_search.fit(X,y)

/usr/local/lib/python3.8/dist-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/usr/local/lib/python3.8/dist-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/usr/local/lib/python3.8/dist-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/usr/local/lib/python3.8/dist-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pa

GridSearchCV(cv=10, estimator=Pipeline(steps=[('clf', LinearRegression())]),
             param_grid=[{'clf': (LinearRegression(),)}, {'clf': (Ridge(),)},
                         {'clf': (XGBRegressor(base_score=None, booster=None,
                                               colsample_bylevel=None,
                                               colsample_bynode=None,
                                               colsample_bytree=None,
                                               enable_categorical=False,
                                               gamma=None, gpu_id=None,
                                               importance_type=None,
                                               interaction_constraints=None,...
                                               min_child_weight=None,
                                               missing=nan,
                                               monotone_constraints=None,
                                               n_estimator

In [42]:
# Printing the best estimator
grid_search.best_estimator_

Pipeline(steps=[('clf', LGBMRegressor())])

In [43]:
# Make predictions 
grid_search_blended_predictions = grid_search.predict(blended_predictions_df[["DT_predictions", "NN_predictions"]])

In [44]:
# Add predictions to a dataframe
grid_search_blended_predictions_df = blended_predictions_df
grid_search_blended_predictions_df["SalePrice"] = grid_search_blended_predictions
grid_search_blended_predictions_df = grid_search_blended_predictions_df.drop(columns=["DT_predictions","NN_predictions"])

# Create CSV with blended predictions
grid_search_blended_predictions_df.to_csv("submission_gridsearch_blended.csv",index=False)

I had high hopes for a meta-model that blended predictions, but it performed worse than I expected.



## Improvements to subsequent models

Below I note some improvements to the models I would implement if I had more time.

- Remove outliers (using something like sklearn's IsolationForest)
- 